In [ ]:
"""
Post-hoc: we need to pickle our entity index for quick loading. This file does so 
"""

In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval
import re
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial import distance 
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
from tqdm import tqdm 
import networkx as nx

In [15]:
#NOTE: very important, which entity categories to keep 
#article showing all entity types below
# https://www.kaggle.com/code/curiousprogrammer/entity-extraction-and-classification-using-spacy
TO_KEEP = ["org","event", "person", "work_of_art", "product"]

OUT_PATH = "/shared/3/projects/newsDiffusion/data/interim/NEREmbedding/NERIndexPickled.pkl"


In [7]:
LOAD_COLS = ["key", "NamedEntities"]

In [8]:
#load in main data source 
print("loading news data")
df = pd.read_csv("/shared/3/projects/newsDiffusion/data/processed/newsData/fullDataWithNERCleaned.tsv",\
                 sep="\t", usecols=LOAD_COLS)

loading news data


In [9]:
#NOTE: used to have embeddings here, but don't need that anymore with current method 
print("parsing")

def cleanList(inList): 
    return [str(re.sub("[^a-zA-Z0-9 ]", "", item).lower()) for item in inList]

def parseList(inStr): 
    split = inStr.split("\'), (\'")
    return [cleanList(item.split("', '")) for item in split]

#parse topics from string to actual list of tuples 
df["NamedEntities"] = df["NamedEntities"].apply(parseList)

print("parsed")

#test out idea for creating reverse mapping 
#how many na vals do we have in "NamedEntities"? 
print(str(sum(df["NamedEntities"].isna())) + " NA values in Named Entities column")
print("Filling with '' instead")
df["NamedEntities"] = df["NamedEntities"].fillna("")

parsing
parsed
0 NA values in Named Entities column
Filling with '' instead


Note: we see below that we have things like "date: week" as named entities. This must be addressed somewhere 

In [11]:
entDf = df.explode("NamedEntities")

In [12]:
entDf[["ent_type","entity"]] = pd.DataFrame(entDf["NamedEntities"].tolist(), index=entDf.index)

In [17]:
entDf[["key", "ent_type", "entity"]].to_pickle(OUT_PATH)

In [24]:
#group articles by their named entities  
entDf = entDf[["ent_type", "entity", "key"]].groupby(by=["ent_type", "entity"]).agg(list)

print(str(len(entDf)) + " rows in entity-grouped df")

7436473 rows in entity-grouped df
